In [57]:
"""
    This is the main script that will create the predictions on test data and save 
    a predictions file.
"""
# import time
from pathlib import Path
import pickle
import pandas as pd

import utils

import pandas as pd
import numpy as np
import os
from node import Node
from datetime import datetime, timedelta
from pathlib import Path
from catboost import CatBoostClassifier, Pool
import sys

# INPUT/OUTPUT PATHS WITHIN THE DOCKER CONTAINER
TEST_DATA_DIR = Path('../../../data/phase_1_v3/fold_0/valid')
TEST_PREDS_FP = Path('../../../data/submission/submission_fuse_metric_sensitive.csv')

class index_dict:
    def __init__(self):
        self.times = self.IDADIK()
        self.indices = []
        self.AD_dex =[]
        self.modes = self.mode()

    class IDADIK:
        def __init__(self):
            self.ID = []
            self.AD = []
            self.IK = []

    class mode:
        def __init__(self):
            self.SK = []
            self.end = []

datalist = []

# Searching for training data within the dataset folder
for file in os.listdir(TEST_DATA_DIR):
    if file.endswith(".csv"):
        datalist.append(os.path.join(TEST_DATA_DIR, file))

# Sort the training data and labels
datalist = sorted(datalist)

# Print the sorted filepath to the training data
print(datalist)

frames = list()

for idx_data in range(len(datalist)):
    detected = index_dict()
    filtered = index_dict()
    
    data_path = datalist[idx_data]
    data = pd.read_csv(data_path)
    
    # Read the objectID from the filename
    filename = data_path.split('/')[-1]
    
    satcat=int(filename.split('.')[0])
    
    # Extracting longitudinal and inclination information from the pandas dataframe
    longitudes = data["Longitude (deg)"]
    inclinations = data["Inclination (deg)"]
    
    # Arbitrary assign start time and end time. Note: SNICT was developed to read in time-stamped data, 
    # however, our training data are not label with a time stamp, hence an arbitrary start and end time
    # are selected
    starttime = datetime.fromisoformat("2023-01-01 00:00:00+00:00")
    endtime = datetime.fromisoformat("2023-07-01 00:00:00+00:00")
    
    # Get std for longitude over a 24 hours window
    lon_std = []
    nodes = []
    steps_per_day = 12
    lon_was_baseline = True
    lon_baseline = 0.03
    
    for i in range(len(data["Longitude (deg)"])):
        if i <= steps_per_day:
            lon_std.append(np.std(data["Longitude (deg)"][0:steps_per_day]))
        else:
            lon_std.append(np.std(data["Longitude (deg)"][i-steps_per_day:i]))
    
    ssEW = Node(satcat=satcat,
                t=starttime,
                index=0,
                ntype="SS",
                signal="EW")
    es = Node(satcat=satcat,
                t=endtime,
                index=len(data["Longitude (deg)"])-1,
                ntype="ES",
                signal="ES",
                mtype="ES")
    
    # Run LS detection
    for i in range(steps_per_day+1,len(lon_std)-steps_per_day):             # if at least 1 day has elapsed since t0
        max_lon_std_24h = np.max(lon_std[i-steps_per_day:i])
        min_lon_std_24h = np.min(lon_std[i-steps_per_day:i])
        A = np.abs(max_lon_std_24h-min_lon_std_24h)/2
        th_ = 0.95*A
    
        # ID detection
        if (lon_std[i]>lon_baseline) & lon_was_baseline:                    # if sd is elevated & last sd was at baseline
            before = np.mean(data["Longitude (deg)"][i-steps_per_day:i])    # mean of previous day's longitudes
            after = np.mean(data["Longitude (deg)"][i:i+steps_per_day])     # mean of next day's longitudes
            # if not temporary noise, then real ID
            if np.abs(before-after)>0.3:                                    # if means are different
                lon_was_baseline = False                                    # the sd is not yet back at baseline
                index = i
                if i < steps_per_day+2:
                    ssEW.mtype = "NK"
                else:
                    detected.times.ID.append(starttime+timedelta(hours=i*2))
        # IK detection
        elif (lon_std[i]<=lon_baseline) & (not lon_was_baseline):           # elif sd is not elevated and drift has already been initialized
            drift_ended = True                                              # toggle end-of-drift boolean 
            for j in range(steps_per_day):                                  # for the next day, check...
                if np.abs(data["Longitude (deg)"][i]-data["Longitude (deg)"][i+j])>0.3:       # if the longitude changes from the current value
                    drift_ended = False                                     # the drift has not ended
            if drift_ended:                                                 # if the drift has ended
                lon_was_baseline = True                                     # the sd is back to baseline
                detected.times.IK.append(starttime+timedelta(hours=i*2))    # save tnow as end-of-drift
                detected.indices.append([index,i])                          # save indices for t-start & t-end
    
        # Last step
        elif (i == (len(lon_std)-steps_per_day-1))\
            &(not lon_was_baseline):
            detected.times.IK.append(starttime+timedelta(hours=i*2))
            detected.indices.append([index,i])
    
        # AD detection
        elif ((lon_std[i]-max_lon_std_24h>th_) or (min_lon_std_24h-lon_std[i]>th_)) & (not lon_was_baseline):          # elif sd is elevated and drift has already been initialized
            if i >= steps_per_day+3:
                detected.times.AD.append(starttime+timedelta(hours=i*2))
                detected.AD_dex.append(i)
    
    def add_node(n):
        nodes[len(nodes)-1].char_mode(
            next_index = n.index,
            lons = longitudes,
            incs = inclinations
        )
        if n.type == "AD":
            nodes[len(nodes)-1].mtype = "NK"
    
        if (nodes[len(nodes)-1].mtype != "NK"):
            filtered.indices.append([nodes[len(nodes)-1].index,n.index])
            filtered.modes.SK.append(nodes[len(nodes)-1].mtype)
            stop_NS = True if n.type == "ID" else False
            filtered.modes.end.append(stop_NS)
        nodes.append(n)
    
    toggle = True
    nodes.append(ssEW)
    if len(detected.times.IK) == 1:
        if len(detected.times.ID) == 1:
            filtered.times.ID.append(detected.times.ID[0])                                  # keep the current ID
            ID = Node(satcat,
                    detected.times.ID[0],
                    index=detected.indices[0][0],
                    ntype='ID',
                    lon=longitudes[detected.indices[0][0]],
                    signal="EW")
            add_node(ID)
        filtered.times.IK.append(detected.times.IK[0]) 
        IK = Node(satcat,
                detected.times.IK[0],
                index=detected.indices[0][1],
                ntype='IK',
                lon=longitudes[detected.indices[0][1]],
                signal="EW")
        apnd = True
        if len(detected.times.AD) == 1:
            AD = Node(satcat,
                      detected.times.AD[0],
                      index=detected.AD_dex[0],
                      ntype="AD",
                      signal="EW")
            add_node(AD)
        elif len(detected.times.AD) == 0:
            pass
        else:
            for j in range(len(detected.times.AD)):
                ad = Node(satcat,
                      detected.times.AD[j],
                      index=detected.AD_dex[j],
                      ntype="AD",
                      signal="EW")
                ad_next = Node(satcat,
                      detected.times.AD[j+1],
                      index=detected.AD_dex[j+1],
                      ntype="AD",
                      signal="EW") \
                    if j < (len(detected.times.AD)-1) else None
                if (ad.t>starttime+timedelta(hours=detected.indices[0][0]*2))&(ad.t<IK.t):
                    if apnd & (ad_next is not None):
                        if ((ad_next.t-ad.t)>timedelta(hours=24)):
                            add_node(ad)
                        else:
                            add_node(ad)
                            apnd = False
                    elif apnd & (ad_next is None):
                        add_node(ad)
                    elif (not apnd) & (ad_next is not None):
                        if ((ad_next.t-ad.t)>timedelta(hours=24)):
                            apnd = True
        if detected.indices[0][1] != (len(lon_std)-steps_per_day-1):
            add_node(IK)    
    
    for i in range(len(detected.times.IK)-1):                                 # for each longitudinal shift detection
        if toggle:                                                            # if the last ID was not discarded
            if ((starttime+timedelta(hours=detected.indices[i+1][0]*2)-detected.times.IK[i])>timedelta(hours=36)):# if the time between the current IK & next ID is longer than 48 hours
                filtered.times.ID.append(detected.times.ID[i])                # keep the current ID
                filtered.times.IK.append(detected.times.IK[i])                # keep the current IK
                ID = Node(satcat,
                        detected.times.ID[i],
                        index=detected.indices[i][0],
                        ntype='ID',
                        lon=longitudes[detected.indices[i][0]],
                        signal="EW")
                add_node(ID)
                IK = Node(satcat,
                        detected.times.IK[i],
                        index=detected.indices[i][1],
                        ntype='IK',
                        lon=longitudes[detected.indices[i][1]],
                        signal="EW")
                apnd = True
                for j in range(len(detected.times.AD)):
                    ad = Node(satcat,
                      detected.times.AD[j],
                      index=detected.AD_dex[j],
                      ntype="AD",
                      signal="EW")
                    ad_next = Node(satcat,
                      detected.times.AD[j+1],
                      index=detected.AD_dex[j+1],
                      ntype="AD",
                      signal="EW") \
                        if j < (len(detected.times.AD)-1) else None
                    if (ad.t>ID.t)&(ad.t<IK.t):
                        if apnd & (ad_next is not None):
                            if ((ad_next.t-ad.t)>timedelta(hours=24)):
                                add_node(ad)
                            else:
                                add_node(ad)
                                apnd = False
                        elif apnd & (ad_next is None):
                            add_node(ad)
                        elif (not apnd) & (ad_next is not None):
                            if ((ad_next.t-ad.t)>timedelta(hours=24)):
                                apnd = True
                if detected.indices[0][1] != (
                    len(lon_std)-steps_per_day-1):
                    add_node(IK)    
                if i == len(detected.times.IK)-2:                             # if the next drift is the last drift
                    filtered.times.ID.append(starttime+timedelta(hours=detected.indices[i+1][0]*2))                    # keep the next ID
                    ID = Node(satcat,
                            starttime+timedelta(hours=detected.indices[i+1][0]*2),
                            index=detected.indices[i+1][0],
                            ntype='ID',
                            lon=longitudes[detected.indices[i+1][0]],
                            signal="EW")
                    add_node(ID)
                    IK = Node(satcat,
                            detected.times.IK[i+1],
                            index=detected.indices[i+1][1],
                            ntype='IK',
                            lon=longitudes[detected.indices[i+1][1]],
                            signal="EW")
                    apnd = True
                    for j in range(len(detected.times.AD)):
                        ad = Node(satcat,
                            detected.times.AD[j],
                            index=detected.AD_dex[j],
                            ntype="AD",
                            signal="EW")
                        ad_next = Node(satcat,
                            detected.times.AD[j+1],
                            index=detected.AD_dex[j+1],
                            ntype="AD",
                            signal="EW") \
                            if j < (len(detected.times.AD)-1) else None
                        if (ad.t>ID.t)&(ad.t<IK.t):
                            if apnd & (ad_next is not None):
                                if ((ad_next.t-ad.t)>timedelta(
                                    hours=24)):
                                    add_node(ad)
                                else:
                                    add_node(ad)
                                    apnd = False
                            elif apnd & (ad_next is None):
                                add_node(ad)
                            elif (not apnd) & (ad_next is not None):
                                if ((ad_next.t-ad.t)>timedelta(
                                    hours=24)):
                                    apnd = True
                    if detected.indices[i][1] != (
                        len(lon_std)-steps_per_day-1):
                        filtered.times.IK.append(detected.times.IK[i+1])      # keep the next IK
                        add_node(IK)
            else:                                                             # if the next ID and the current IK are 48 hours apart or less
                ID = Node(satcat,
                        detected.times.ID[i],
                        index=detected.indices[i][0],
                        ntype='ID',
                        lon=longitudes[detected.indices[i][0]],
                        signal="EW")                                          # keep the current ID
                add_node(ID)
                AD = Node(satcat,
                        detected.times.IK[i],
                        index=detected.indices[i][1],
                        ntype='AD',
                        lon=longitudes[detected.indices[i][1]],
                        signal="EW")                                          # change the current IK to an AD
                IK = Node(satcat,
                        detected.times.IK[i+1],
                        index=detected.indices[i+1][1],
                        ntype='IK',
                        lon=longitudes[detected.indices[i+1][1]],
                        signal="EW")                                          # exchange the current IK for the next one
                add_node(AD)
                apnd = True
                for j in range(len(detected.times.AD)):
                    ad = Node(satcat,
                      detected.times.AD[j],
                      index=detected.AD_dex[j],
                      ntype="AD",
                      signal="EW")
                    ad_next = Node(satcat,
                      detected.times.AD[j+1],
                      index=detected.AD_dex[j+1],
                      ntype="AD",
                      signal="EW") \
                        if j < (len(detected.times.AD)-1) else None
                    if (ad.t>ID.t)&(ad.t<IK.t):
                        if apnd & (ad_next is not None):
                            if ((ad_next.t-ad.t)>timedelta(hours=24)):
                                add_node(ad)
                            else:
                                add_node(ad)
                                apnd = False
                        elif apnd & (ad_next is None):
                            add_node(ad)
                        elif (not apnd) & (ad_next is not None):
                            if ((ad_next.t-ad.t)>timedelta(hours=24)):
                                apnd = True
                if detected.indices[0][1] != (
                    len(lon_std)-steps_per_day-1):
                    add_node(IK)    
                filtered.times.ID.append(detected.times.ID[i])
                filtered.times.AD.append(detected.times.IK[i])
                filtered.times.IK.append(detected.times.IK[i+1])
                toggle = False                                                # skip the redundant drift
        else:
            toggle = True
    add_node(es)

    ssNS = Node(
            satcat=satcat,
            t=starttime,
            index=0,
            ntype="SS",
            signal="NS",
            mtype="NK")

    for j in range(len(filtered.indices)):
        indices = filtered.indices[j]
        first = True if indices[0] == 0 else False
        times = []
        dexs = []
        inc = inclinations[indices[0]:indices[1]].to_numpy()
        t = np.arange(indices[0],indices[1])*2
        rate = (steps_per_day/(indices[1]-indices[0]))*(np.max(inc)-np.min(inc))
        XIPS_inc_per_day = 0.0005 #0.035/30
        if (rate < XIPS_inc_per_day) and (indices[0] < steps_per_day) and (indices[1] > steps_per_day):
            if filtered.modes.end[j]:
                nodes.append(Node(
                    satcat=satcat,
                    t=starttime+timedelta(hours=indices[1]*2),
                    index=indices[1],
                    ntype="ID",
                    signal="NS",
                    mtype="NK"
                ))
                
            ssNS.mtype = filtered.modes.SK[j]
        elif (rate < XIPS_inc_per_day):
            nodes.append(Node(
                satcat=satcat,
                t=starttime+timedelta(hours=indices[1]*2),
                index=indices[1],
                ntype="IK",
                signal="NS",
                mtype=filtered.modes.SK[j]
            ))
            if filtered.modes.end[j]:
                nodes.append(Node(
                    satcat=satcat,
                    t=starttime+timedelta(hours=indices[1]*2),
                    index=indices[1],
                    ntype="ID",
                    signal="NS",
                    mtype="NK"
                ))
        else:
            dt = [0.0]
            for i in range(len(inc)-1):
                dt.append((inc[i+1]-inc[i])/(2*60*60))
            prev = 1.0

            for i in range(len(dt)-1):
                if np.abs(dt[i])> 5.5e-7:
                    times.append(starttime+timedelta(hours=float(t[i])))
                    dexs.append(i+indices[0])
                    if (np.abs(np.mean(inc[0:i])-np.mean(inc[i:len(inc)]))/(np.std(inc[0:i])+sys.float_info.epsilon))/prev < 1.0:
                        if first and len(times)==2:
                            ssNS.mtype = filtered.modes.SK[0]
                            first = False
                    elif len(times)==2:
                        first = False
                    prev = np.abs(np.mean(inc[0:i])-np.mean(inc[i:len(inc)]))/(np.std(inc[0:i])+sys.float_info.epsilon)

            if len(times)>0:
                nodes.append(Node(
                    satcat=satcat,
                    t=times[0],
                    index=dexs[0],
                    ntype="IK",
                    signal="NS",
                    mtype=filtered.modes.SK[j]
                ))
                ssNS.mtype = "NK"
                if filtered.modes.end[j]:
                    nodes.append(Node(
                        satcat=satcat,
                        t=starttime+timedelta(hours=indices[1]*2),
                        index=indices[1],
                        ntype="ID",
                        signal="NS",
                        mtype="NK"
                    ))
            elif filtered.indices[0][0] == 0:
                ssNS.mtype = filtered.modes.SK[0]
            else:
                ssNS.mtype = "NK"
    nodes.append(ssNS)
    nodes.sort(key=lambda x: x.t)
    
    # Convert timestamp back into timeindex and format the output to the correct format in a pandas dataframe
    ObjectID_list = []
    TimeIndex_list = []
    Direction_list = []
    Node_list = []
    Type_list = []
    for i in range(len(nodes)):
        ObjectID_list.append(nodes[i].satcat)
        TimeIndex_list.append(int(((nodes[i].t-starttime).days*24+(nodes[i].t-starttime).seconds/3600)/2))
        Direction_list.append(nodes[i].signal)
        Node_list.append(nodes[i].type)
        Type_list.append(nodes[i].mtype)
    
    # Initialize data of lists. 
    data = {'ObjectID': ObjectID_list, 
            'TimeIndex': TimeIndex_list,
            'Direction': Direction_list, 
            'Node': Node_list,
            'Type': Type_list} 
    
    # Create the pandas DataFrame 
    prediction_temp = pd.DataFrame(data) 
    frames.append(prediction_temp)

# Create the pandas DataFrame 
prediction_HR = pd.concat(frames)
# prediction_HR.rename(columns={"Node": "Node_HP", "Type": "Type_HP"},inplace=  True)
divisor = 12
prediction_HR['Quotient'], _ = zip(*prediction_HR['TimeIndex'].map(lambda x: divmod(x, divisor)))  

# INPUT/OUTPUT PATHS WITHIN THE DOCKER CONTAINER
TRAINED_MODEL_DIR = Path('./trained_model/')
# TEST_DATA_DIR = Path('../../../data/phase_1_v3/fold_0/valid')
# TEST_PREDS_FP = Path('../../../data/submission/submission_metric_sensitive_xgboost.csv')


# Rest of configuration, specific to this submission
feature_cols = [
    "Eccentricity",
    "Semimajor Axis (m)",
    "Inclination (deg)",
    "RAAN (deg)",
    "Argument of Periapsis (deg)",
    "True Anomaly (deg)",
    "Latitude (deg)",
    "Longitude (deg)",
    "Altitude (m)",
    "X (m)",
    "Y (m)",
    "Z (m)",
    "Vx (m/s)",
    "Vy (m/s)",
    "Vz (m/s)"
]

lag_steps = 5

test_data, updated_feature_cols = utils.tabularize_data(
    TEST_DATA_DIR, feature_cols, lag_steps=lag_steps)

# Load the trained models (don't use the utils module, use pickle)
model_EW = pickle.load(open(TRAINED_MODEL_DIR / 'XG_model_EW.pkl', 'rb'))
model_NS = pickle.load(open(TRAINED_MODEL_DIR / 'XG_model_NS.pkl', 'rb'))
le_EW = pickle.load(open(TRAINED_MODEL_DIR / 'XG_le_EW.pkl', 'rb'))
le_NS = pickle.load(open(TRAINED_MODEL_DIR / 'XG_le_NS.pkl', 'rb'))

# Make predictions on the test data for EW
test_data['Predicted_EW'] = le_EW.inverse_transform(
    model_EW.predict(test_data[updated_feature_cols])
)

# Make predictions on the test data for NS
test_data['Predicted_NS'] = le_NS.inverse_transform(
    model_NS.predict(test_data[updated_feature_cols])
)
list_unique_ids = test_data["ObjectID"].unique().tolist()
test_results_frames = []
for idx in list_unique_ids:
    try:
        selected = test_data[test_data["ObjectID"]==idx][['TimeIndex', 'ObjectID', 
                    'Predicted_EW', 'Predicted_NS']]
        # Detect the first occurrence of a new value by comparing each row to the next one
        selected['first_occurrence'] = (selected['Predicted_EW'] != selected['Predicted_EW'].shift(1)) | (selected.index == 0)

        # Filter the DataFrame to keep only the first occurrences
        first_occurrences_df_ew = selected[selected['first_occurrence']]

        # Drop the auxiliary column if it's no longer needed
        first_occurrences_df_ew = first_occurrences_df_ew.drop(columns=['first_occurrence'])

        # Now first_occurrences_df contains only the rows where a new sequence starts
        first_occurrences_df_ew = first_occurrences_df_ew[first_occurrences_df_ew["Predicted_EW"]!="XX-XX"][["TimeIndex", "ObjectID", "Predicted_EW"]]    
        # Detect the first occurrence of a new value by comparing each row to the next one
        selected['first_occurrence'] = (selected['Predicted_NS'] != selected['Predicted_NS'].shift(1)) | (selected.index == 0)

        # Filter the DataFrame to keep only the first occurrences
        first_occurrences_df_ns = selected[selected['first_occurrence']]

        # Drop the auxiliary column if it's no longer needed
        first_occurrences_df_ns = first_occurrences_df_ns.drop(columns=['first_occurrence'])

        # Now first_occurrences_df contains only the rows where a new sequence starts
        first_occurrences_df_ns = first_occurrences_df_ns[first_occurrences_df_ns["Predicted_NS"]!="XX-XX"][["TimeIndex", "ObjectID", "Predicted_NS"]]   

        final_df = utils.convert_classifier_output_sep(first_occurrences_df_ew,first_occurrences_df_ns)     

        final_df['Quotient'], _ = zip(*final_df['TimeIndex'].map(lambda x: divmod(x, divisor)))    

        test_results_frames.append(final_df)
    except:
        print(idx)

prediction_XG = pd.concat(test_results_frames)

test_data.drop(columns=["Predicted_EW","Predicted_NS"], inplace=True)

# Load the trained models (don't use the utils module, use pickle)
model_EW = CatBoostClassifier()
model_EW.load_model(TRAINED_MODEL_DIR / 'CB_model_EW')
model_NS = CatBoostClassifier()
model_NS.load_model(TRAINED_MODEL_DIR / 'CB_model_NS')
le_EW = pickle.load(open(TRAINED_MODEL_DIR / 'CB_le_EW.pkl', 'rb'))
le_NS = pickle.load(open(TRAINED_MODEL_DIR / 'CB_le_NS.pkl', 'rb'))

# Make predictions on the test data for EW
test_data['Predicted_EW'] = le_EW.inverse_transform(
    model_EW.predict(test_data[updated_feature_cols])
)

# Make predictions on the test data for NS
test_data['Predicted_NS'] = le_NS.inverse_transform(
    model_NS.predict(test_data[updated_feature_cols])
)

list_unique_ids = test_data["ObjectID"].unique().tolist()
test_results_frames = []
divisor = 12
for idx in list_unique_ids:
    try:
        selected = test_data[test_data["ObjectID"]==idx][['TimeIndex', 'ObjectID', 
                    'Predicted_EW', 'Predicted_NS']]
        # Detect the first occurrence of a new value by comparing each row to the next one
        selected['first_occurrence'] = (selected['Predicted_EW'] != selected['Predicted_EW'].shift(1)) | (selected.index == 0)

        # Filter the DataFrame to keep only the first occurrences
        first_occurrences_df_ew = selected[selected['first_occurrence']]

        # Drop the auxiliary column if it's no longer needed
        first_occurrences_df_ew = first_occurrences_df_ew.drop(columns=['first_occurrence'])

        # Now first_occurrences_df contains only the rows where a new sequence starts
        first_occurrences_df_ew = first_occurrences_df_ew[first_occurrences_df_ew["Predicted_EW"]!="XX-XX"][["TimeIndex", "ObjectID", "Predicted_EW"]]    
        # Detect the first occurrence of a new value by comparing each row to the next one
        selected['first_occurrence'] = (selected['Predicted_NS'] != selected['Predicted_NS'].shift(1)) | (selected.index == 0)

        # Filter the DataFrame to keep only the first occurrences
        first_occurrences_df_ns = selected[selected['first_occurrence']]

        # Drop the auxiliary column if it's no longer needed
        first_occurrences_df_ns = first_occurrences_df_ns.drop(columns=['first_occurrence'])

        # Now first_occurrences_df contains only the rows where a new sequence starts
        first_occurrences_df_ns = first_occurrences_df_ns[first_occurrences_df_ns["Predicted_NS"]!="XX-XX"][["TimeIndex", "ObjectID", "Predicted_NS"]]   

        final_df = utils.convert_classifier_output_sep(first_occurrences_df_ew,first_occurrences_df_ns)     

        final_df['Quotient'], _ = zip(*final_df['TimeIndex'].map(lambda x: divmod(x, divisor)))    

        test_results_frames.append(final_df)
    except:
        print(idx)

prediction_CB = pd.concat(test_results_frames)

test_data.drop(columns=["Predicted_EW","Predicted_NS"], inplace=True)

# Load the trained models (don't use the utils module, use pickle)
model_EW = pickle.load(open(TRAINED_MODEL_DIR / 'RF_model_EW.pkl', 'rb'))
model_NS = pickle.load(open(TRAINED_MODEL_DIR / 'RF_model_NS.pkl', 'rb'))
le_EW = pickle.load(open(TRAINED_MODEL_DIR / 'RF_le_EW.pkl', 'rb'))
le_NS = pickle.load(open(TRAINED_MODEL_DIR / 'RF_le_NS.pkl', 'rb'))

# Make predictions on the test data for EW
test_data['Predicted_EW'] = le_EW.inverse_transform(
    model_EW.predict(test_data[updated_feature_cols])
)

# Make predictions on the test data for NS
test_data['Predicted_NS'] = le_NS.inverse_transform(
    model_NS.predict(test_data[updated_feature_cols])
)

list_unique_ids = test_data["ObjectID"].unique().tolist()
test_results_frames = []
divisor = 12
for idx in list_unique_ids:
    try:
        selected = test_data[test_data["ObjectID"]==idx][['TimeIndex', 'ObjectID', 
                    'Predicted_EW', 'Predicted_NS']]
        # Detect the first occurrence of a new value by comparing each row to the next one
        selected['first_occurrence'] = (selected['Predicted_EW'] != selected['Predicted_EW'].shift(1)) | (selected.index == 0)

        # Filter the DataFrame to keep only the first occurrences
        first_occurrences_df_ew = selected[selected['first_occurrence']]

        # Drop the auxiliary column if it's no longer needed
        first_occurrences_df_ew = first_occurrences_df_ew.drop(columns=['first_occurrence'])

        # Now first_occurrences_df contains only the rows where a new sequence starts
        first_occurrences_df_ew = first_occurrences_df_ew[first_occurrences_df_ew["Predicted_EW"]!="XX-XX"][["TimeIndex", "ObjectID", "Predicted_EW"]]    
        # Detect the first occurrence of a new value by comparing each row to the next one
        selected['first_occurrence'] = (selected['Predicted_NS'] != selected['Predicted_NS'].shift(1)) | (selected.index == 0)

        # Filter the DataFrame to keep only the first occurrences
        first_occurrences_df_ns = selected[selected['first_occurrence']]

        # Drop the auxiliary column if it's no longer needed
        first_occurrences_df_ns = first_occurrences_df_ns.drop(columns=['first_occurrence'])

        # Now first_occurrences_df contains only the rows where a new sequence starts
        first_occurrences_df_ns = first_occurrences_df_ns[first_occurrences_df_ns["Predicted_NS"]!="XX-XX"][["TimeIndex", "ObjectID", "Predicted_NS"]]   

        final_df = utils.convert_classifier_output_sep(first_occurrences_df_ew,first_occurrences_df_ns)     

        final_df['Quotient'], _ = zip(*final_df['TimeIndex'].map(lambda x: divmod(x, divisor)))    

        test_results_frames.append(final_df)
    except:
        print(idx)

prediction_RF = pd.concat(test_results_frames)

['../../../data/phase_1_v3/fold_0/valid/1002.csv', '../../../data/phase_1_v3/fold_0/valid/1013.csv', '../../../data/phase_1_v3/fold_0/valid/1019.csv', '../../../data/phase_1_v3/fold_0/valid/1022.csv', '../../../data/phase_1_v3/fold_0/valid/1025.csv', '../../../data/phase_1_v3/fold_0/valid/1033.csv', '../../../data/phase_1_v3/fold_0/valid/1040.csv', '../../../data/phase_1_v3/fold_0/valid/1051.csv', '../../../data/phase_1_v3/fold_0/valid/1052.csv', '../../../data/phase_1_v3/fold_0/valid/1055.csv', '../../../data/phase_1_v3/fold_0/valid/1056.csv', '../../../data/phase_1_v3/fold_0/valid/1065.csv', '../../../data/phase_1_v3/fold_0/valid/1068.csv', '../../../data/phase_1_v3/fold_0/valid/107.csv', '../../../data/phase_1_v3/fold_0/valid/1073.csv', '../../../data/phase_1_v3/fold_0/valid/1076.csv', '../../../data/phase_1_v3/fold_0/valid/108.csv', '../../../data/phase_1_v3/fold_0/valid/1083.csv', '../../../data/phase_1_v3/fold_0/valid/11.csv', '../../../data/phase_1_v3/fold_0/valid/110.csv', '../

/home/rabay/anaconda3/envs/pol-challenge/lib/python3.9/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


234
3
9
238
225
232
246
245
236
242
17
235


/home/rabay/anaconda3/envs/pol-challenge/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rabay/anaconda3/envs/pol-challenge/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


234
9
225
232
245
236
242
235
234
9
225
232
246
245
236
242
17
235


In [1568]:
data_gt = pd.read_csv("../../../data/phase_1_v3/fold_0/valid_labels.csv")
divisor = 12
data_gt['Quotient'], _ = zip(*data_gt['TimeIndex'].map(lambda x: divmod(x, divisor))) 

In [1569]:
resultant_frame = {
                    "ObjectID":	[],
                    "TimeIndex":[],	
                    "Direction": [],	
                    "Node":	[],
                    "Type": []
}

In [2851]:
resultant_frame = {
                    "ObjectID":	[],
                    "TimeIndex":[],	
                    "Direction": [],	
                    "Node":	[],
                    "Type": []
}
lst_objectid = data_gt["ObjectID"].unique().tolist()
for objectid in lst_objectid:
  print("ObjectID {}".format(objectid))
  selected_HR = prediction_HR[prediction_HR["ObjectID"]==objectid]
  selected_HR["model"] = "HR"
  selected_HR_EW = selected_HR[selected_HR["Direction"]=="EW"]
  selected_HR_NS = selected_HR[selected_HR["Direction"]=="NS"]
  selected_XG = prediction_XG[prediction_XG["ObjectID"]==objectid]
  selected_XG["model"] = "XG"
  selected_XG_EW = selected_XG[selected_XG["Direction"]=="EW"]
  selected_XG_NS = selected_XG[selected_XG["Direction"]=="NS"]
  selected_CB = prediction_CB[prediction_CB["ObjectID"]==objectid]
  selected_CB["model"] = "CB"
  selected_CB_EW = selected_CB[selected_CB["Direction"]=="EW"]
  selected_CB_NS = selected_CB[selected_CB["Direction"]=="NS"]
  selected_RF = prediction_RF[prediction_RF["ObjectID"]==objectid]
  selected_RF["model"] = "RF"
  selected_RF_EW = selected_RF[selected_RF["Direction"]=="EW"]
  selected_RF_NS = selected_RF[selected_RF["Direction"]=="NS"]
  new_frame_EW = []
  new_frame_EW.append(selected_RF_EW)
  new_frame_EW.append(selected_XG_EW)
  new_frame_EW.append(selected_CB_EW)
  new_frame_EW.append(selected_HR_EW)
  new_frame_EW_pd = pd.concat(new_frame_EW)  
  new_frame_NS = []
  new_frame_NS.append(selected_RF_NS)
  new_frame_NS.append(selected_XG_NS)
  new_frame_NS.append(selected_CB_NS)
  new_frame_NS.append(selected_HR_NS)
  new_frame_NS_pd = pd.concat(new_frame_NS)    
  list_quotient_EW = new_frame_EW_pd["Quotient"].unique().tolist()
  list_quotient_NS = new_frame_NS_pd["Quotient"].unique().tolist()
  for quotient_EW in list_quotient_EW:
      print("EW unique quotient {}".format(quotient_EW))
      selected_new_frame_EW_pd = new_frame_EW_pd[new_frame_EW_pd["Quotient"]==quotient_EW]
      # if quotient_EW == 0:
      #   resultant_frame["ObjectID"].append(selected_new_frame_EW_pd.ObjectID.tolist()[0])
      #   resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_EW_pd.TimeIndex.tolist())))
      #   resultant_frame["Direction"].append("EW")   
      #   resultant_frame["Type"].append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["model"]=="HR"].Type.tolist()[0]) 
      #   resultant_frame["Node"].append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["model"]=="HR"].Node.tolist()[0])          
      # else:
      lst_unique_node = selected_new_frame_EW_pd.Node.unique().tolist()
      lst_unique_type = selected_new_frame_EW_pd.Type.unique().tolist()
      if len(selected_new_frame_EW_pd.model.tolist()) == 1:
        # pass
        if selected_new_frame_EW_pd.model.tolist()[0] == "HR" and min(selected_new_frame_EW_pd.TimeIndex.tolist())==0:
          resultant_frame["ObjectID"].append(selected_new_frame_EW_pd.ObjectID.tolist()[0])
          resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_EW_pd.TimeIndex.tolist())))
          resultant_frame["Direction"].append("EW")
          resultant_frame["Node"].append(selected_new_frame_EW_pd.Node.tolist()[0])
          resultant_frame["Type"].append(selected_new_frame_EW_pd.Type.tolist()[0])
      else:
        if len(lst_unique_node)==1 and len(lst_unique_type)==1:
          resultant_frame["ObjectID"].append(selected_new_frame_EW_pd.ObjectID.tolist()[0])
          resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_EW_pd.TimeIndex.tolist())))
          resultant_frame["Direction"].append("EW")
          resultant_frame["Node"].append(lst_unique_node[0])
          resultant_frame["Type"].append(lst_unique_type[0])                      
        else:
          if len(list_quotient_EW) ==1:
            resultant_frame["ObjectID"].append(selected_new_frame_EW_pd.ObjectID.tolist()[0])
            resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_EW_pd.TimeIndex.tolist())))
            resultant_frame["Direction"].append("EW")   
            resultant_frame["Type"].append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["model"]=="HR"].Type.tolist()[0]) 
            resultant_frame["Node"].append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["model"]=="HR"].Node.tolist()[0])           
          else:
            resultant_frame["ObjectID"].append(selected_new_frame_EW_pd.ObjectID.tolist()[0])
            resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_EW_pd.TimeIndex.tolist())))
            resultant_frame["Direction"].append("EW")          
            lst_count = []
            lst_sum = []
            for idx in lst_unique_type:
              lst_sum.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==idx].groupby("TimeIndex").sum().index.tolist()[0])
              lst_count.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==idx].groupby("TimeIndex").count().Type.tolist()[0]) 
            max_value_count = min(lst_count)
            max_indices_count = [index for index, value in enumerate(lst_count) if value == max_value_count]    
            max_value = min(max_indices_count)
            max_indices = [index for index, value in enumerate(max_indices_count) if value == max_value]               
            resultant_frame["Type"].append(lst_unique_type[max_indices[0]])     
            lst_count = []
            lst_sum = []
            for idx in lst_unique_node:
              lst_sum.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Node"]==idx].groupby("TimeIndex").sum().index.tolist()[0])
              lst_count.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Node"]==idx].groupby("TimeIndex").count().Node.tolist()[0])         
            max_value_count = min(lst_count)
            max_indices_count = [index for index, value in enumerate(lst_count) if value == max_value_count]
            max_value = min(max_indices_count)
            max_indices = [index for index, value in enumerate(max_indices_count) if value == max_value] 
            resultant_frame["Node"].append(lst_unique_node[max_indices[0]])     
  for quotient_NS in list_quotient_NS:
      print("NS unique quotient {}".format(quotient_NS))
      selected_new_frame_NS_pd = new_frame_NS_pd[new_frame_NS_pd["Quotient"]==quotient_NS]
      # if quotient_NS == 0:
      #   resultant_frame["ObjectID"].append(selected_new_frame_NS_pd.ObjectID.tolist()[0])
      #   resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_NS_pd.TimeIndex.tolist())))
      #   resultant_frame["Direction"].append("NS")   
      #   resultant_frame["Type"].append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["model"]=="HR"].Type.tolist()[0]) 
      #   resultant_frame["Node"].append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["model"]=="HR"].Node.tolist()[0])         
      # else:
      lst_unique_node = selected_new_frame_NS_pd.Node.unique().tolist()
      lst_unique_type = selected_new_frame_NS_pd.Type.unique().tolist()
      if len(selected_new_frame_NS_pd.model.tolist()) == 1:
        # pass
        if selected_new_frame_NS_pd.model.tolist()[0] == "HR" and min(selected_new_frame_NS_pd.TimeIndex.tolist())==0:
          resultant_frame["ObjectID"].append(selected_new_frame_NS_pd.ObjectID.tolist()[0])
          resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_NS_pd.TimeIndex.tolist())))
          resultant_frame["Direction"].append("NS")
          resultant_frame["Node"].append(selected_new_frame_NS_pd.Node.tolist()[0])
          resultant_frame["Type"].append(selected_new_frame_NS_pd.Type.tolist()[0])
      else:
        if len(lst_unique_node)==1 and len(lst_unique_type)==1:
          resultant_frame["ObjectID"].append(selected_new_frame_NS_pd.ObjectID.tolist()[0])
          resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_NS_pd.TimeIndex.tolist())))
          resultant_frame["Direction"].append("NS")
          resultant_frame["Node"].append(lst_unique_node[0])
          resultant_frame["Type"].append(lst_unique_type[0])                      
        else:
          if len(list_quotient_NS) ==1:
            resultant_frame["ObjectID"].append(selected_new_frame_NS_pd.ObjectID.tolist()[0])
            resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_NS_pd.TimeIndex.tolist())))
            resultant_frame["Direction"].append("NS")   
            resultant_frame["Type"].append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["model"]=="HR"].Type.tolist()[0]) 
            resultant_frame["Node"].append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["model"]=="HR"].Node.tolist()[0])           
          else:          
            resultant_frame["ObjectID"].append(selected_new_frame_NS_pd.ObjectID.tolist()[0])
            resultant_frame["TimeIndex"].append(int(np.mean(selected_new_frame_NS_pd.TimeIndex.tolist())))
            resultant_frame["Direction"].append("NS")          
            lst_count = []
            lst_sum = []
            for idx in lst_unique_type:
              lst_sum.append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["Type"]==idx].groupby("TimeIndex").sum().index.tolist()[0])
              lst_count.append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["Type"]==idx].groupby("TimeIndex").count().Type.tolist()[0]) 
            max_value_count = min(lst_count)
            max_indices_count = [index for index, value in enumerate(lst_count) if value == max_value_count]    
            max_value = min(max_indices_count)
            max_indices = [index for index, value in enumerate(max_indices_count) if value == max_value]               
            resultant_frame["Type"].append(lst_unique_type[max_indices[0]])     
            lst_count = []
            lst_sum = []
            for idx in lst_unique_node:
              lst_sum.append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["Node"]==idx].groupby("TimeIndex").sum().index.tolist()[0])
              lst_count.append(selected_new_frame_NS_pd[selected_new_frame_NS_pd["Node"]==idx].groupby("TimeIndex").count().Node.tolist()[0])         
            max_value_count = min(lst_count)
            max_indices_count = [index for index, value in enumerate(lst_count) if value == max_value_count]
            max_value = min(max_indices_count)
            max_indices = [index for index, value in enumerate(max_indices_count) if value == max_value] 
            resultant_frame["Node"].append(lst_unique_node[max_indices[0]])                           

ObjectID 3
EW unique quotient 0
NS unique quotient 0
ObjectID 6
EW unique quotient 0
NS unique quotient 0
ObjectID 7
EW unique quotient 0
NS unique quotient 0
ObjectID 9
EW unique quotient 0
NS unique quotient 0
NS unique quotient 41
ObjectID 11
EW unique quotient 0
NS unique quotient 0
ObjectID 15
EW unique quotient 0
NS unique quotient 0
ObjectID 17
EW unique quotient 0
NS unique quotient 0
NS unique quotient 11
ObjectID 20
EW unique quotient 0
NS unique quotient 0
ObjectID 31
EW unique quotient 0
NS unique quotient 0
ObjectID 43
EW unique quotient 0
NS unique quotient 0
ObjectID 44
EW unique quotient 0
NS unique quotient 0
ObjectID 45
EW unique quotient 0
NS unique quotient 0
ObjectID 58
EW unique quotient 0
NS unique quotient 0
ObjectID 63
EW unique quotient 0
NS unique quotient 0
ObjectID 76
EW unique quotient 0
NS unique quotient 0
NS unique quotient 23
ObjectID 99
EW unique quotient 0
NS unique quotient 0
NS unique quotient 22
ObjectID 107
EW unique quotient 0
EW unique quotient

/tmp/ipykernel_908521/3824463369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_HR["model"] = "HR"
/tmp/ipykernel_908521/3824463369.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_CB["model"] = "CB"
/tmp/ipykernel_908521/3824463369.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

ObjectID 149
EW unique quotient 0
EW unique quotient 142
EW unique quotient 167
EW unique quotient 130
NS unique quotient 0
NS unique quotient 142
NS unique quotient 130
NS unique quotient 167
ObjectID 155
EW unique quotient 0
EW unique quotient 153
NS unique quotient 0
NS unique quotient 11
ObjectID 157
EW unique quotient 0
NS unique quotient 0
NS unique quotient 11
ObjectID 169
EW unique quotient 0
EW unique quotient 142
NS unique quotient 0
NS unique quotient 142
NS unique quotient 143
ObjectID 171
EW unique quotient 0
EW unique quotient 17
EW unique quotient 18
EW unique quotient 68
NS unique quotient 0
NS unique quotient 8
NS unique quotient 17
ObjectID 175
EW unique quotient 0
EW unique quotient 99
EW unique quotient 150
NS unique quotient 0
NS unique quotient 8
NS unique quotient 99
ObjectID 176
EW unique quotient 0
EW unique quotient 108
EW unique quotient 116
EW unique quotient 167
NS unique quotient 0
NS unique quotient 108
NS unique quotient 116
ObjectID 179
EW unique quotie

/tmp/ipykernel_908521/3824463369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_HR["model"] = "HR"
/tmp/ipykernel_908521/3824463369.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_XG["model"] = "XG"
/tmp/ipykernel_908521/3824463369.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

EW unique quotient 0
NS unique quotient 0
ObjectID 235
EW unique quotient 0
NS unique quotient 0
ObjectID 236
EW unique quotient 0
NS unique quotient 0
ObjectID 238
EW unique quotient 0
NS unique quotient 0
ObjectID 242
EW unique quotient 0
NS unique quotient 0
ObjectID 245
EW unique quotient 0
NS unique quotient 0
ObjectID 246
EW unique quotient 110
EW unique quotient 168
EW unique quotient 0
EW unique quotient 107
EW unique quotient 116
EW unique quotient 129
EW unique quotient 131
EW unique quotient 132
NS unique quotient 107
NS unique quotient 0
NS unique quotient 180
ObjectID 261
EW unique quotient 0
EW unique quotient 61
EW unique quotient 139
EW unique quotient 169
EW unique quotient 140
EW unique quotient 170
NS unique quotient 0
NS unique quotient 61
NS unique quotient 171
ObjectID 269
EW unique quotient 0
EW unique quotient 24
EW unique quotient 98
EW unique quotient 109
EW unique quotient 99
EW unique quotient 100
EW unique quotient 110
NS unique quotient 0
NS unique quotien

...

/tmp/ipykernel_908521/3824463369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_HR["model"] = "HR"
/tmp/ipykernel_908521/3824463369.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_XG["model"] = "XG"
/tmp/ipykernel_908521/3824463369.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

NS unique quotient 95
ObjectID 1634
EW unique quotient 0
NS unique quotient 0
NS unique quotient 85
ObjectID 1638
EW unique quotient 0
NS unique quotient 0
NS unique quotient 105
ObjectID 1646
EW unique quotient 0
NS unique quotient 0
ObjectID 1656
EW unique quotient 0
NS unique quotient 0
ObjectID 1659
EW unique quotient 0
NS unique quotient 0
ObjectID 1662
EW unique quotient 0
NS unique quotient 0
NS unique quotient 118
ObjectID 1664
EW unique quotient 0
NS unique quotient 0
ObjectID 1676
EW unique quotient 0
NS unique quotient 0
ObjectID 1684
EW unique quotient 0
NS unique quotient 0
NS unique quotient 117
ObjectID 1685
EW unique quotient 0
NS unique quotient 0
NS unique quotient 123
NS unique quotient 4
ObjectID 1687
EW unique quotient 0
NS unique quotient 0
ObjectID 1688
EW unique quotient 0
NS unique quotient 0
ObjectID 1692
EW unique quotient 0
NS unique quotient 0
NS unique quotient 103
ObjectID 1695
EW unique quotient 0
NS unique quotient 0
NS unique quotient 84
ObjectID 1699


/tmp/ipykernel_908521/3824463369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_HR["model"] = "HR"
/tmp/ipykernel_908521/3824463369.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_XG["model"] = "XG"
/tmp/ipykernel_908521/3824463369.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

ObjectID 1821
EW unique quotient 0
NS unique quotient 0
ObjectID 1824
EW unique quotient 0
NS unique quotient 0
ObjectID 1826
EW unique quotient 0
NS unique quotient 0
ObjectID 1828
EW unique quotient 0
NS unique quotient 0
ObjectID 1833
EW unique quotient 0
NS unique quotient 0
ObjectID 1836
EW unique quotient 0
NS unique quotient 0
NS unique quotient 5
ObjectID 1840
EW unique quotient 0
NS unique quotient 0
ObjectID 1842
EW unique quotient 0
NS unique quotient 0
ObjectID 1844
EW unique quotient 0
NS unique quotient 0
ObjectID 1848
EW unique quotient 0
NS unique quotient 0
ObjectID 1856
EW unique quotient 0
NS unique quotient 0
ObjectID 1868
EW unique quotient 0
NS unique quotient 0
ObjectID 1869
EW unique quotient 0
NS unique quotient 0
ObjectID 1872
EW unique quotient 0
NS unique quotient 0
ObjectID 1876
EW unique quotient 0
NS unique quotient 0
ObjectID 1881
EW unique quotient 0
NS unique quotient 0
ObjectID 1883
EW unique quotient 0
NS unique quotient 0
ObjectID 1890
EW unique quo

/tmp/ipykernel_908521/3824463369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_HR["model"] = "HR"
/tmp/ipykernel_908521/3824463369.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_XG["model"] = "XG"
/tmp/ipykernel_908521/3824463369.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [2852]:
selected_new_frame_EW_pd[selected_new_frame_EW_pd["model"]=="HR"].Type.tolist()[0]

'CK'

In [2853]:
resultant_frame_pd = pd.DataFrame(resultant_frame)

In [2983]:
idx = 19
resultant_frame_pd[resultant_frame_pd["ObjectID"]==lst_objectid[idx]]

,ObjectID,TimeIndex,Direction,Node,Type
45,112,0,EW,SS,NK
46,112,207,EW,IK,HK
47,112,194,EW,AD,NK
48,112,0,NS,SS,NK
49,112,231,NS,IK,HK


In [2984]:
resultant_frame_pd.to_csv("../../../data/submission/participant.csv")

In [2985]:
lst_objectid = data_gt["ObjectID"].unique().tolist()
# objectidx = 51
# objectid = lst_objectid[objectidx]
objectid = lst_objectid[idx]

In [2986]:
import evaluation
evaluator = evaluation.NodeDetectionEvaluator(data_gt[data_gt["ObjectID"]==objectid],resultant_frame_pd[resultant_frame_pd["ObjectID"]==lst_objectid[idx]],6)
precision, recall, f2, rmse = evaluator.score()
print(f'Precision for the validation set: {precision:.2f}')
print(f'Recall for the validation set: {recall:.2f}')
print(f'F2 for the validation set: {f2:.2f}')
print(f'RMSE for the train set: {rmse:.2f}')

Total TPs: 4
Total FPs: 1
Total FNs: 1
Total Distances: [0, 1, -6, 0]
Precision for the validation set: 0.80
Recall for the validation set: 0.80
F2 for the validation set: 0.80
RMSE for the train set: 3.04


In [2987]:
data_gt_selected = data_gt[data_gt["ObjectID"]==objectid]
data_gt_selected[data_gt_selected["Direction"]=="EW"]

,Unnamed: 0,ObjectID,TimeIndex,Direction,Node,Type,Quotient
69,7444,112,0,EW,SS,NK,0
71,7446,112,200,EW,AD,NK,16
72,7447,112,206,EW,IK,HK,17


In [2988]:
data_gt_selected[data_gt_selected["Direction"]=="NS"]

,Unnamed: 0,ObjectID,TimeIndex,Direction,Node,Type,Quotient
70,7445,112,0,NS,SS,NK,0
73,7448,112,239,NS,IK,HK,19


In [2935]:
data_gt_selected[data_gt_selected["Direction"]=="NS"]

,Unnamed: 0,ObjectID,TimeIndex,Direction,Node,Type,Quotient
44,4460,76,0,NS,SS,CK,0


In [2936]:
selected_HR = prediction_HR[prediction_HR["ObjectID"]==objectid]

In [2937]:
selected_HR["model"] = "HR"

/tmp/ipykernel_908521/2326444870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_HR["model"] = "HR"


In [2938]:
selected_HR_EW = selected_HR[selected_HR["Direction"]=="EW"]

In [2939]:
selected_HR_EW

,ObjectID,TimeIndex,Direction,Node,Type,Quotient,model
0,76,0,EW,SS,CK,0,HR


In [2940]:
selected_HR_NS = selected_HR[selected_HR["Direction"]=="NS"]

In [2941]:
selected_XG = prediction_XG[prediction_XG["ObjectID"]==objectid]

In [2942]:
selected_XG["model"] = "XG"

/tmp/ipykernel_908521/3541905448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_XG["model"] = "XG"


In [2943]:
selected_XG_EW = selected_XG[selected_XG["Direction"]=="EW"]

In [2944]:
selected_XG_NS = selected_XG[selected_XG["Direction"]=="NS"]

In [2945]:
selected_CB = prediction_CB[prediction_CB["ObjectID"]==objectid]

In [2946]:
selected_CB["model"] = "CB"

/tmp/ipykernel_908521/4279518435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_CB["model"] = "CB"


In [2947]:
selected_CB_EW = selected_CB[selected_CB["Direction"]=="EW"]

In [2948]:
selected_CB_NS = selected_CB[selected_CB["Direction"]=="NS"]

In [2949]:
selected_RF = prediction_RF[prediction_RF["ObjectID"]==objectid]

In [2950]:
selected_RF["model"] = "RF"

/tmp/ipykernel_908521/3305876843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_RF["model"] = "RF"


In [2951]:
selected_RF_EW = selected_RF[selected_RF["Direction"]=="EW"]

In [2952]:
selected_RF_NS = selected_RF[selected_RF["Direction"]=="NS"]

In [2953]:
new_frame_EW = []
new_frame_EW.append(selected_RF_EW)
new_frame_EW.append(selected_XG_EW)
new_frame_EW.append(selected_CB_EW)
new_frame_EW.append(selected_HR_EW)
new_frame_EW_pd = pd.concat(new_frame_EW)


In [2954]:
new_frame_EW_pd.sort_values("Quotient")

,ObjectID,TimeIndex,Direction,Node,Type,Quotient,model
0,76,0,EW,SS,CK,0,RF
0,76,0,EW,SS,CK,0,XG
0,76,0,EW,SS,CK,0,CB
0,76,0,EW,SS,CK,0,HR


In [2955]:
new_frame_NS = []
new_frame_NS.append(selected_RF_NS)
new_frame_NS.append(selected_XG_NS)
new_frame_NS.append(selected_CB_NS)
new_frame_NS.append(selected_HR_NS)
new_frame_NS_pd = pd.concat(new_frame_NS)


In [2956]:
new_frame_NS_pd.sort_values("Quotient")

,ObjectID,TimeIndex,Direction,Node,Type,Quotient,model
1,76,0,NS,SS,NK,0,RF
1,76,0,NS,SS,NK,0,XG
1,76,0,NS,SS,NK,0,CB
2,76,1,NS,SS,CK,0,CB
3,76,2,NS,SS,NK,0,CB
1,76,0,NS,SS,NK,0,HR
2,76,286,NS,IK,CK,23,HR


In [2722]:
idx_quotient = 0
list_quotient = new_frame_EW_pd["Quotient"].unique().tolist()
list_quotient[idx_quotient]

0

In [2595]:
selected_new_frame_EW_pd = new_frame_EW_pd[new_frame_EW_pd["Quotient"]==list_quotient[idx_quotient]]

In [2378]:
selected_new_frame_EW_pd.head()

,ObjectID,TimeIndex,Direction,Node,Type,Quotient,model
0,15,0,EW,SS,EK,0,RF
1,15,1,EW,SS,HK,0,RF
0,15,0,EW,SS,EK,0,XG
1,15,1,EW,SS,HK,0,XG
0,15,0,EW,SS,EK,0,CB


In [2379]:
selected_new_frame_EW_pd.Node.tolist()[0]

'SS'

In [2380]:
lst_unique_node = selected_new_frame_EW_pd.Node.unique().tolist()

In [2381]:
lst_unique_node

['SS']

In [1700]:
lst_unique_type = selected_new_frame_EW_pd.Type.unique().tolist()

In [1701]:
lst_unique_type

['NK']

In [1702]:
# if mumltiple output then below

In [1703]:
idx

'CK'

In [1685]:
# selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==idx].groupby("TimeIndex").sum().index.tolist()[0]

In [1686]:
lst_count = []
lst_sum = []
for idx in lst_unique_node:
  lst_sum.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Node"]==idx].groupby("TimeIndex").sum().index.tolist()[0])
  lst_count.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Node"]==idx].groupby("TimeIndex").count().Node.tolist()[0])

In [1687]:
max_value_count = max(lst_count)
max_indices_count = [index for index, value in enumerate(lst_count) if value == max_value_count]

In [1688]:
max_value = max(max_indices_count)
max_indices = [index for index, value in enumerate(max_indices_count) if value == max_value]

In [1689]:
lst_unique_node[max_indices[0]]

'SS'

In [1690]:
lst_count = []
lst_sum = []
for idx in lst_unique_type:
  lst_sum.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==idx].groupby("TimeIndex").sum().index.tolist()[0])
  lst_count.append(selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==idx].groupby("TimeIndex").count().Type.tolist()[0])


In [1691]:
max_value_count = max(lst_count)
max_indices_count = [index for index, value in enumerate(lst_count) if value == max_value_count]

In [1692]:
max_value = max(max_indices_count)
max_indices = [index for index, value in enumerate(max_indices_count) if value == max_value]

In [1693]:
lst_unique_type[max_indices[0]]

'EK'

In [1053]:
lst_sum[max_indices[0]]

1527

In [561]:
max_indices_count[max_indices[0]]

0

In [415]:
lst_sum[0]

540

In [416]:
lst_sum[1]

546

In [417]:
lst_unique_type[1]

'EK'

In [418]:
idx = 2
lst_unique_type[idx]

IndexError: list index out of range

In [148]:
selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==lst_unique_type[idx]].groupby("TimeIndex").sum().index.tolist()[0]

0

In [149]:
selected_new_frame_EW_pd[selected_new_frame_EW_pd["Type"]==lst_unique_type[idx]].groupby("TimeIndex").count().Type.tolist()[0]

1

In [16]:
data_gt = pd.read_csv("../../../data/phase_1_v3/fold_0/valid_labels.csv")
divisor = 12
data_gt['Quotient'], _ = zip(*data_gt['TimeIndex'].map(lambda x: divmod(x, divisor))) 

In [40]:
data_gt_selected = data_gt[data_gt["ObjectID"]==419]

In [41]:
data_gt_selected[data_gt_selected["Direction"]=="EW"]

,Unnamed: 0,ObjectID,TimeIndex,Direction,Node,Type,Quotient
418,7881,419,0,EW,SS,NK,0
420,7883,419,1369,EW,AD,NK,114
421,7884,419,1375,EW,IK,EK,114
423,7886,419,1909,EW,ID,NK,159


In [42]:
data_gt_selected[data_gt_selected["Direction"]=="NS"]

,Unnamed: 0,ObjectID,TimeIndex,Direction,Node,Type,Quotient
419,7882,419,0,NS,SS,NK,0
422,7885,419,1396,NS,IK,CK,116
424,7887,419,1909,NS,ID,NK,159
